

# Quick Visualization for Hyperparameter Optimization Analysis

Optuna provides various visualization features in :mod:`optuna.visualization` to analyze optimization results visually.

This tutorial walks you through this module by visualizing the history of lightgbm model for breast cancer dataset.

For visualizing multi-objective optimization (i.e., the usage of :func:`optuna.visualization.plot_pareto_front`),
please refer to the tutorial of `multi_objective`.

<div class="alert alert-info"><h4>Note</h4><p>By using [Optuna Dashboard](https://github.com/optuna/optuna-dashboard), you can also check the optimization history,
   hyperparameter importances, hyperparameter relationships, etc. in graphs and tables.
   Please make your study persistent using `RDB backend <rdb>` and execute following commands to run Optuna Dashboard.

```console
$ pip install optuna-dashboard
$ optuna-dashboard sqlite:///example-study.db
```
   Please check out [the GitHub repository](https://github.com/optuna/optuna-dashboard) for more details.

   .. list-table::
      :header-rows: 1

      * - Manage Studies
        - Visualize with Interactive Graphs
      * - .. image:: https://user-images.githubusercontent.com/5564044/205545958-305f2354-c7cd-4687-be2f-9e46e7401838.gif
        - .. image:: https://user-images.githubusercontent.com/5564044/205545965-278cd7f4-da7d-4e2e-ac31-6d81b106cada.gif</p></div>


In [11]:
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from experiment.exp import Exp_XGBoost
from data.argparser import args_parsing
from data.data_loader import Dataset_XGB
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from utils.postprocessing import ProcessedResult
import optuna
from utils.metrics import LinEx, LinLin, weighted_RMSE, RMSE

# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

SEED = 42

np.random.seed(SEED)

In [2]:
import argparse
import os
import yaml
import torch

import numpy as np
from datetime import datetime


now = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

parser = argparse.ArgumentParser(description='[XGBoost] Forecasting')

# Main arguements
parser.add_argument('--data', type=str, required=False, default='SRL_NEG_00_04', help='data')
parser.add_argument('--model', type=str, required=False, default='xgboost',help='model of experiment, options: []')

parser.add_argument('--loss', type=str, default='rmse',help='customized loss functions, one of [w_rmse, linex, linlin, rmse]')

parser.add_argument('--w_rmse_weight', type=float, default=5,help='weighted parameter for weighted rmse loss function')
parser.add_argument('--linex_weight', type=float, default=0.05,help='weighted parameter for linear-exponential loss function')
parser.add_argument('--linlin_weight', type=float, default=0.1,help='weighted parameter for linlin / pinball loss function')

parser.add_argument('--input_len', type=int, default=30, help='input sequence length')
parser.add_argument('--target_len', type=int, default=1, help='prediction length')

parser.add_argument('--timestamp', type=str, default=now)

parser.add_argument('--root_path', type=str, default= 'data\\processed\\SRL\\', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='SRL_NEG_00_04.csv', help='data file')    
parser.add_argument('--features', type=str, default='S', help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--cols', type=str, nargs='+', help='external col names from the data files as the additional input features (not including target)')

parser.add_argument('--scale', type=str, default='standard', help='forecasting task, options: [standard, minmax, none]')
parser.add_argument('--target', type=str, default='capacity_price', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='d', help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')

# Model arguments

parser.add_argument('--n_estimators', type=int, default=100, help='The number of gradient boosted trees.')
parser.add_argument('--max_depth', type=int, default=3, help='Maximum depth of a tree.')
parser.add_argument('--min_child_weight', type=float, default=1.0, help='Minimum sum of instance weight needed in a child.')
parser.add_argument('--learning_rate', type=float, default=0.1, help='Boosting learning rate.')
parser.add_argument('--subsample', type=float, default=1.0, help='Subsample ratio of the training instances.')
parser.add_argument('--colsample_bytree', type=float, default=1.0, help='Subsample ratio of columns when constructing each tree.')
parser.add_argument('--reg_gamma', type=float, default=0.0, help='L2 regularization.')
parser.add_argument('--reg_alpha', type=float, default=0.0, help='L1 regularization.')

# FOR TUNING
parser.add_argument('--tune_num_samples', type=int, default=5, help='Number of sample interations in hyperparameter tuning')

args = parser.parse_args("")

args.root_path = os.path.normpath(args.root_path)

args.freq = args.freq[-1:]

# Pass default values for external data incorporation
if args.features == 'MS' and args.cols is None:
    args.cols = ['gas', 'coal']

print('Args in experiment:')
print(args)
print('')


# with open("args.yaml", "w") as f:
#     yaml.dump(args, f)

Args in experiment:
Namespace(data='SRL_NEG_00_04', model='xgboost', loss='rmse', w_rmse_weight=5, linex_weight=0.05, linlin_weight=0.1, input_len=30, target_len=1, timestamp='06-07-2023_23-19-43', root_path='data\\processed\\SRL', data_path='SRL_NEG_00_04.csv', features='S', cols=None, scale='standard', target='capacity_price', freq='d', n_estimators=100, max_depth=3, min_child_weight=1.0, learning_rate=0.1, subsample=1.0, colsample_bytree=1.0, reg_gamma=0.0, reg_alpha=0.0, tune_num_samples=5)



Define the objective function.



In [2]:
# def objective(trial):
#     data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
#     train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
#     dtrain = lgb.Dataset(train_x, label=train_y)
#     dvalid = lgb.Dataset(valid_x, label=valid_y)

#     param = {
#         "objective": "binary",
#         "metric": "auc",
#         "verbosity": -1,
#         "boosting_type": "gbdt",
#         "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
#         "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
#         "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
#     }

#     # Add a callback for pruning.
#     pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
#     gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback])

#     preds = gbm.predict(valid_x)
#     pred_labels = np.rint(preds)
#     accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
#     return accuracy

In [5]:
args.root_path

'data\\processed\\SRL'

In [7]:
Dataset_XGB(root_path='./data/processed/SRL/', 
                          data=args.data)

In [43]:
def objective(trial):
    
    # param = {
    #     "objective": "binary",
    #     "metric": "auc",
    #     "verbosity": -1,
    #     "boosting_type": "gbdt",
    #     "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
    #     "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
    #     "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    # }
    
    # SEARCH SPACE
    
    args.n_estimators = trial.suggest_int("n_estimators", 10, 100)
    args.max_depth = trial.suggest_int("max_depth", 3, 12)
    # subsample=self.args.subsample,
    # min_child_weight=self.args.min_child_weight,
    # colsample_bytree=self.args.colsample_bytree,
    args.learning_rate = trial.suggest_float("learning_rate", 1e-1, 1e0, log=True)
    # tree_method="hist"
    
    # TRAINING
    
    train_data = Dataset_XGB(
        root_path=args.root_path,
        data_path=args.data_path,
        flag='train',
        input_len=args.input_len,
        target_len=args.target_len,
        features='S',
        target='capacity_price',
        timeenc=1,
        freq='d',
        scale='standard',
        cols=None
    )
    
    model = xgb.XGBRegressor(
        n_estimators=args.n_estimators,
        max_depth=args.max_depth,
        subsample=args.subsample,
        min_child_weight=args.min_child_weight,
        colsample_bytree=args.colsample_bytree,
        learning_rate=args.learning_rate,
        objective='reg:squarederror', # either 'reg:squarederror' or custom objective
        eval_metric=mean_absolute_error, # function, e.g. from sk.metrics
        tree_method="hist"
    )

    model = MultiOutputRegressor(model)
    
    trained_model = model.fit(
    # np.concatenate((train_data.matrix_x, train_data.matrix_mark), 1), 
    train_data.matrix_x, 
    train_data.matrix_y)
    
    vali_data = Dataset_XGB(
        root_path=args.root_path,
        data_path=args.data_path,
        flag='val',
        input_len=args.input_len,
        target_len=args.target_len,
        features='S',
        target='capacity_price',
        timeenc=1,
        freq='d',
        scale='standard',
        cols=None
    )
    
    preds = model.predict(vali_data.matrix_x)
    trues = vali_data.matrix_y
        
    result = ProcessedResult(preds, trues, args=args, data=vali_data)
    
    loss = RMSE(result.pred, result.true)
    revenue = result.predict_revenue(result.pred)
    
    # # Add a callback for pruning.
    # pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
    # gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback])

    # preds = gbm.predict(valid_x)
    # pred_labels = np.rint(preds)
    # accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return loss, revenue

In [44]:
study = optuna.create_study(
    directions=['minimize', 'maximize'],
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
)
study.optimize(objective, n_trials=100, timeout=600)

[I 2023-07-07 00:24:19,066] A new study created in memory with name: no-name-ad4f20a6-ef18-4a92-99ca-e23b2c05b49c
[I 2023-07-07 00:24:20,719] Trial 0 finished with values: [80.47061575506926, 0.0] and parameters: {'n_estimators': 44, 'max_depth': 12, 'learning_rate': 0.008471801418819975}. 
[I 2023-07-07 00:24:21,066] Trial 1 finished with values: [115.64696277896796, 0.0] and parameters: {'n_estimators': 64, 'max_depth': 4, 'learning_rate': 4.207053950287933e-05}. 
[I 2023-07-07 00:24:21,703] Trial 2 finished with values: [111.70617708972051, 0.0] and parameters: {'n_estimators': 15, 'max_depth': 11, 'learning_rate': 0.002537815508265664}. 
[I 2023-07-07 00:24:22,013] Trial 3 finished with values: [14.438143925034131, 1527.02] and parameters: {'n_estimators': 74, 'max_depth': 3, 'learning_rate': 0.07579479953348005}. 
[I 2023-07-07 00:24:22,614] Trial 4 finished with values: [115.44046084454098, 0.0] and parameters: {'n_estimators': 85, 'max_depth': 5, 'learning_rate': 5.3370327626039

## Plot functions
Visualize the optimization history. See :func:`~optuna.visualization.plot_optimization_history` for the details.



In [45]:
optuna.visualization.plot_pareto_front(study, target_names=["loss", "revenue"])

In [51]:
plot_optimization_history(study, target=lambda t: t.values[1])

Visualize the learning curves of the trials. See :func:`~optuna.visualization.plot_intermediate_values` for the details.



In [17]:
plot_intermediate_values(study)

[W 2023-07-06 23:38:03,494] You need to set up the pruning feature to utilize `plot_intermediate_values()`


Visualize high-dimensional parameter relationships. See :func:`~optuna.visualization.plot_parallel_coordinate` for the details.



In [53]:
plot_parallel_coordinate(study,target=lambda t: t.values[1])

Select parameters to visualize.



In [ ]:
plot_parallel_coordinate(study, params=["bagging_freq", "bagging_fraction"])

Visualize hyperparameter relationships. See :func:`~optuna.visualization.plot_contour` for the details.



In [55]:
plot_contour(study, target=lambda t: t.values[1])

Select parameters to visualize.



In [ ]:
plot_contour(study, params=["bagging_freq", "bagging_fraction"])

Visualize individual hyperparameters as slice plot. See :func:`~optuna.visualization.plot_slice` for the details.



In [ ]:
plot_slice(study)

Select parameters to visualize.



In [ ]:
plot_slice(study, params=["bagging_freq", "bagging_fraction"])

Visualize parameter importances. See :func:`~optuna.visualization.plot_param_importances` for the details.



In [ ]:
plot_param_importances(study)

Learn which hyperparameters are affecting the trial duration with hyperparameter importance.



In [ ]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.duration.total_seconds(), target_name="duration"
)

Visualize empirical distribution function. See :func:`~optuna.visualization.plot_edf` for the details.



In [ ]:
plot_edf(study)